In [2]:

import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import math
from glob import glob
from pathlib import Path
import joblib
import pickle
import itertools
from tqdm.auto import tqdm
import re
import torch
import vllm

# Initialize LLM and Tokenizer
llm = vllm.LLM(
    "Qwen/Qwen2.5-Math-7B-Instruct",  # Ensure this model is supported
    tensor_parallel_size=1,  # or 4 based on available resources
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
)
tokenizer = llm.get_tokenizer()

WARNING 10-29 12:55:01 config.py:1668] Casting torch.bfloat16 to torch.float16.
WARNING 10-29 12:55:05 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-29 12:55:05 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='Qwen/Qwen2.5-Math-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-Math-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=O

model-00002-of-00004.safetensors:  68%|######8   | 2.64G/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:  78%|#######8  | 3.08G/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:  70%|#######   | 2.71G/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:  98%|#########8| 3.49G/3.56G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-29 12:57:26 model_runner.py:1067] Loading model weights took 14.2418 GB
INFO 10-29 12:59:07 gpu_executor.py:122] # GPU blocks: 753, # CPU blocks: 4681
INFO 10-29 12:59:07 gpu_executor.py:126] Maximum concurrency for 4096 tokens per request: 2.94x


In [ ]:
import csv
import re

from IPython import get_ipython
from tqdm.auto import tqdm

def solve_problem(id, problem):
    # Prompt engineering
    sampling_params = vllm.SamplingParams(
      temperature=0.00,
      max_tokens=1024
    )
    prompt = f"Let's solve the following AIMO2 problem step by step:\n{problem}\n\nStep 1: "

    # Generate solution using LLM
    solution_steps = llm.generate(prompt, sampling_params=sampling_params, use_tqdm=False)

    # Extract Python code from solution steps
    python_code = extract_python_code(solution_steps)

    # Run Python code using PythonREPL
    result = PythonREPL.run(python_code)

    return result

def extract_python_code(solution_steps):
    # Extract Python code from the generated solution steps
    # Implement this function based on the format of the generated solution
    # Return the extracted Python code
    
    # Assuming the Python code is enclosed in ```python and ``` delimiters
    python_code = re.search(r"```python(.*?)```", solution_steps, re.DOTALL)
    
    if python_code:
        return python_code.group(1).strip()
    else:
        return None

def main():
    with open('reference.csv', 'r') as file:
        reader = csv.DictReader(file)
        answers = []

        for row in reader:
            id = row['id']
            problem = row['problem']

            print(f"Solving problem {id}:")
            print(problem)

            solution = solve_problem(id, problem)

            print(f"Your solution: {solution}")
            print()

            answers.append({'id': id, 'solution': solution})

        # Save answers to a CSV file
        with open('answers.csv', 'w') as outfile:
            fieldnames = ['id', 'solution']
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(answers)

tokenizer = llm.get_tokenizer()
ipython = get_ipython()
PythonREPL = ipython.run_cell_magic("capture", "--no-stderr", "from IPython import InteractiveShell; PythonREPL = InteractiveShell.instance(); PythonREPL.run_cell('%load_ext autoreload'); PythonREPL.run_cell('%autoreload 2')")
main()

Solving problem 057f8a:
Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?


In [4]:
!pip install tqdm
!pip install csv
!pip install IPython

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv
